In [30]:
    try:
        from .markdown import renderer, CommonMark
        from .environment import environment
    except:
        from markdown import renderer, CommonMark
        from environment import environment

        import IPython
    import types, sys, IPython, asyncio, trio, traceback, ipykernel, traitlets, nbconvert, jinja2, operator, toolz.curried as toolz, tokenize, io, textwrap
    ip = IPython.get_ipython()
    import vdom

    import toolz.curried as toolz

    from dataclasses import dataclass, field

    tangle = renderer

    try:
        from .expressions import evaluate_expression, evaluate_single_expressions
    except:
        from expressions import evaluate_expression, evaluate_single_expressions
        

In [54]:
    import contextlib

In [55]:
    c = contextlib.ExitStack()

In [56]:
    class AsyncMixin:
        def __enter__(self):
            IPython.display.display(self)
            self._capturer = IPython.utils.capture.capture_output()
            self.captured = self._capturer.__enter__()
            return self
                
        def __exit__(self, *exc):
            self._capturer.__exit__(*exc)
            self.capture or self.captured.show()
            if all(e is None for e in exc): 
                trio.run(self.aeval)


In [57]:
    @dataclass
    class Weave(AsynxMixin):
        """A contextmanager to initialize and finalize a weave step.  
        """
        source: str = ""
        block: str = ""
        inline: bool = False
        expressions: tuple = field(default_factory=tuple)
        template: bool = True
        markdown: bool = True
        namespace: dict = field(default_factory=dict, repr=False)
        display: bool = True
        capture: bool = False
        update: bool = True
        environment: jinja2.Environment = field(default=environment)
        ip: IPython.core.interactiveshell.InteractiveShell = field(default_factory=get_ipython)        
        
        def __post_init__(self):
            if self.markdown:
                self.block, self.expressions = renderer(self.source)
            else:
                self.block = self.block or self.source                
            self.namespace = self.namespace or self.ip.user_ns
            
            if self.display is True:
                self.display = IPython.core.display.DisplayHandle()
                        
        async def eval(self, *futures): 
            async with trio.open_nursery() as nursery:
                self.inline and self.expressions and IPython.display.display(IPython.display.Markdown('---'))
                for expression in self.expressions:
                    nursery.start_soon(
                        evaluate_expression, self.ip, expression, 
                        self.namespace, self.namespace)
                self.template and nursery.start_soon(self.update)
        
        

        async def update(self):
            try:
                self._should_display and self.display.update(
                    IPython.display.Markdown(self.render()))
            except: __import__('traceback').print_exc()
        
        def render(self):
            return self.environment.from_string(self.source).render(**self.namespace)
                
        @property
        def _should_display(self):
            if not self.display: return 
            if not self.source.splitlines()[0].strip(): return 
            if self.source.lstrip().startswith('%%'): return 
            if textwrap.dedent(self.source).strip() == self.block.strip(): return 
            if self.display is False: return 
            return True
                
                    
        def _ipython_display_(self):
            self._should_display and self.display.display(IPython.display.Markdown(self.source))

In [58]:
    class SyncMixin:
                    
        def __enter__(self):
            self._capturer = IPython.utils.capture.capture_output()
            self.captured = self._capturer.__enter__()
            return self
                
        def __exit__(self, *exc):
            self._capturer.__exit__(*exc)
            
            IPython.display.display(IPython.display.Markdown(self.render()))
            if exc == (None, None, None):
                self.capture or self.captured.show()
            else:
                traceback.print_last()
                
            self.expressions and IPython.display.display(IPython.display.Markdown('---'))
            for expression in self.expressions:
                IPython.display.display(vdom.code('>>> '+expression))
                IPython.display.display(evaluate_single_expressions(self.ip, expression, self.namespace, self.namespace))


In [59]:
class WeaveSync(SyncMixin, Weave): ...